In [10]:
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
from torch_geometric.datasets import TUDataset
from tqdm.notebook import tqdm

# %pip install --upgrade jupyter ipywidgets
%jupyter nbextension enable --py widgetsnbextension # removed !pip on the recommendation of a comment.


UsageError: Line magic function `%jupyter` not found.


In [11]:
dataset = TUDataset(root="datasets/", name='IMDB-BINARY')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data = dataset[0]
model = Node2Vec(dataset[0].edge_index, embedding_dim=64, walk_length=20, 
                 context_size=10, walks_per_node=10,
                 num_negative_samples=1, p=1, q=1, sparse=True).to(device)


In [12]:
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)  # data loader to speed the train 
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)  # initzialize the optimizer 

In [13]:
for p, n in loader:
    print(p)

TypeError: cannot pickle 'PyCapsule' object

In [7]:
def train():
    model.train()  # put model in train model
    total_loss = 0
    for pos_rw, neg_rw in tqdm(loader):
        optimizer.zero_grad()  # set the gradients to 0
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))  # compute the loss for the batch
        loss.backward()
        optimizer.step()  # optimize the parameters
        total_loss += loss.item()
    return total_loss / len(loader)


for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

all_vectors = ""
for tensor in model(torch.arange(data.num_nodes, device=device)):
    s = "\t".join([str(value) for value in tensor.detach().cpu().numpy()])
    all_vectors += s + "\n"
# save the vectors
with open("vectors.txt", "w") as f:
    f.write(all_vectors)
# save the labels
with open("labels.txt", "w") as f:
    f.write("\n".join([str(label) for label in data.y.numpy()]))

TypeError: cannot pickle 'PyCapsule' object